### > Criando dados artificiais para teste:

In [32]:
#import dask_cudf #dask permite excução em multiplas gpu
#gdf = dask_cudf.read_csv("/path/to/my/data-*.csv")  # Your CSV files can be larger than GPU memory
#and you can use multiple GPUs to process this data

import cudf #cria o dataframe na gpu
from cuml.datasets import make_blobs
import time

X, y = make_blobs(n_samples=15000, centers=3, n_features=5)


tabela_artificial_cudf = cudf.DataFrame(dict(x=X[:,0], y=X[:,1],label=y))
coluna_resposta_cudf = tabela_artificial_cudf['label']

tabela_artificial_cudf

,x,y,label
0,1.381891,-2.298180,2.0
1,1.428387,-3.233190,2.0
2,3.668118,-7.568221,1.0
3,5.194736,4.380275,0.0
4,4.432270,-7.989467,1.0
...,...,...,...
14995,4.920474,-9.510699,1.0
14996,1.336689,5.658974,0.0
14997,2.522857,4.292397,0.0
14998,3.988511,6.646187,0.0


### > Implementando o algoritmo KMeans na GPU:

In [33]:
from cuml.cluster import KMeans

#inicializando as variáveis
inicio = time.time()
tabela_artificial_cudf = cudf.DataFrame(dict(x=X[:,0], y=X[:,1]))
coluna_resposta_cudf = y

#Treinando o modelo pelo algoritmo KMeans
kmeans_cuml = KMeans(n_clusters = 3, random_state = 0)
kmeans_cuml.fit(tabela_artificial_cudf)

#capturando as amostras preditas
labels = kmeans_cuml.labels_
#confrontando com os valores reais
acertos = sum(coluna_resposta_cudf == labels)

#imprimindo o tempo de execução
print("Tempo de execução: " ,time.time() - inicio) 
#imprimindo a acurácia do modelo
print("Resultado: %d de %d amostras foram corretamente preditas." % (acertos, coluna_resposta_cudf.size))
print('Acurácia: {0:0.2f}'. format(acertos/float(coluna_resposta_cudf.size)))

del coluna_resposta_cudf #destroi objeto liberando memoria da gpu
del tabela_artificial_cudf 
del labels

Tempo de execução:  3.7299327850341797
Resultado: 4928 de 15000 amostras foram corretamente preditas.
Acurácia: 0.33


### > Buscando informações da GPU:

In [24]:
# traz informaçõs da gpu
!nvidia-smi
# comando para finalizar o processo na gpu pelo pid
#nvidia-smi | grep 'python' | awk '{ print $3 }' | xargs -n1 kill -9


Mon Oct 10 18:59:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76.02    Driver Version: 517.48       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0  On |                  N/A |
| 30%   45C    P8    18W / 270W |   3550MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Implementando a biblioteca CUDA do compilador NUMBA:

In [37]:
# NUMBA é um compilador python de alta performance para funções matemáticas e array baseada em decorators(@xxx).
# CUDA é uma biblioteca do NUMBA para execução em GPU(Placa de vídeo) 
from numba import cuda #numba aceita apenas alguns tipos de variaveis como float e integer

@cuda.jit(device=True) #funcao executada na gpu e só pode ser chamada de dentro da gpu por uma kernel ou outra device function, possui retorno
def a_device_function(a, b):
    return a + b

@cuda.jit #função kernel é chamada pela cpu e executada na gpu. jit: compilado em tempo de execução
def increment_by_one(algum_array):
    a_device_function(a,b)
    pos = cuda.grid(1) #grid1 é o array unidimensional que contémos id das threads que são executadas simutanemanete
    if pos < algum_array.size: # garante que o index estara no range do array passado como argumento
        algum_array[pos] += 1
        
@cuda.jit #função kernel é chamada pela cpu e executada na gpu. jit: compilado em tempo de execução
def teste_imprime_pos():
    pos = cuda.grid(1) #grid1 é o array unidimensional de threads que são executadas simutanemanete
    print(pos)



#128 é o número de blocos por grid e 1024 o número de threads por bloco,
#multiplica-se os dois para obter o número total de treahds
#o limite desses números variam de acordo com a qualidade da GPU

#teste_imprime_pos[128,1024]() #ira executar 128*1024= 131.072 threads simultaneamente imprimindo o id da thread(pos)
teste_imprime_pos[10,2]() #ira executar 10*2= 20 threads simultaneamente imprimindo o id da thread(pos)
#perceba que os id´s não são impressos em ordem pois são executados simultâneamente.

device = cuda.get_current_device()
my_sms = getattr(device, 'MULTIPROCESSOR_COUNT')
my_cc = device.compute_capability

print(device)
print("GPU compute capability: " , my_cc)
print("GPU total number of SMs: " , my_sms)

16
17
4
5
18
19
6
7
14
15
2
3
10
11
12
13
0
1
8
9
<CUDA device 0 'b'NVIDIA GeForce RTX 3070''>
GPU compute capability:  (8, 6)
GPU total number of SMs:  46
